# Big Data for Health (CSE6250) 
Goal: Sepsis prediction using MIMIC III Data

Author: Zhensheng Wang
         
Created: 10/19/2021

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# from pyspark.sql import SparkSession
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import roc_auc_score
from tableone import TableOne
import lightgbm as lgbm
from hyperopt import fmin, hp, tpe, STATUS_OK, Trials
import os, gc
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= os.path.join(os.getcwd(), "bdfh.json")
from google.cloud import bigquery
bqclient = bigquery.Client()

## Query the data

In [2]:
# Download query results. (change to your own query code)

def query_data_save(tbl):
    
    if f"{tbl}.csv" not in os.listdir('./Data'):
        df_query = bqclient.query(f"select * from cdcproject.BDFH.{tbl}").result().to_dataframe()
        df_query.to_csv(f'Data/{tbl}.csv', index=False)
        print(f"{tbl} created, saved and loaded!")
    else:
        df_query = pd.read_csv(f"Data/{tbl}.csv")
        print(f"{tbl} loaded!")
    return df_query
    

In [2]:
sepsis = query_data_save('sepsis')
nonsepsis = query_data_save('Nonsepsis')
angus_sepsis = query_data_save('angus_sepsis')
sirs = query_data_save('sirs')
sofa = query_data_save('sofa')
cohort = query_data_save('Cohort')
# WBC = pd.read_csv('Data/WBC.csv')
# Creatinine = pd.read_csv('Data/WBC.csv')
# Anion_gap  = pd.read_csv('Data/Anion_gap.csv')
# Sodium  = pd.read_csv('Data/Sodium.csv')
# Hemoglobin  = pd.read_csv('Data/Hemoglobin.csv')
# Lactate  = pd.read_csv('Data/Lactate.csv')
# Potassium  = pd.read_csv('Data/Potassium.csv')
# Urea_Nitrogen = pd.read_csv('Data/Urea_Nitrogen.csv')
# Glucose = pd.read_csv('Data/Glucose.csv')


In [5]:
lab_measures = query_data_save('Hourly_Lab_Measures')


Hourly_Lab_Measures created, saved and loaded!


# Copy the dataframe

In [3]:
print(f"Number of sepsis patients: {len(sepsis)}")
print(f"Number of control patients: {len(nonsepsis)}")
# sepsis.head(5)

# df_sepsis = sepsis.copy()
# df_nonsepsis = nonsepsis.copy()

Number of sepsis patients: 5035
Number of control patients: 42836


## Clean the patient data

In [3]:
def data_clean(df):

    # race recode
    cond_white = df['ETHNICITY'].str.contains('WHITE')
    cond_black = df['ETHNICITY'].str.contains('BLACK')
    cond_asian = df['ETHNICITY'].str.contains('ASIAN')
    cond_hispa = df['ETHNICITY'].str.contains('HISPANIC')

    df.loc[cond_white, 'ETHNICITY'] = 'WHITE'
    df.loc[cond_black, 'ETHNICITY'] = 'BLACK'
    df.loc[cond_asian, 'ETHNICITY'] = 'ASIAN'
    df.loc[cond_hispa, 'ETHNICITY'] = 'HISPANIC'
    df.loc[~(cond_white | cond_black | cond_asian | cond_hispa), 'ETHNICITY'] = 'OTHER'

    df['ETHNICITY'] = df['ETHNICITY'].apply(lambda x: x[0] + x[1:].lower())

    # marital status recode
    cond_other_marital = df['MARITAL_STATUS'].str.contains('SEPARATED|LIFE PARTNER', na = False)
    cond_unknown_marital = df['MARITAL_STATUS'].str.contains('UNKNOWN', na = False) | df['MARITAL_STATUS'].isna()

    df.loc[cond_other_marital, 'MARITAL_STATUS'] = 'OTHER'
    df.loc[cond_unknown_marital, 'MARITAL_STATUS'] = 'UNKNOWN'
    
    df['MARITAL_STATUS'] = df['MARITAL_STATUS'].apply(lambda x: x[0] + x[1:].lower())
    df['gender'] = df['gender'].apply(lambda x: 'Female' if x == 'F' else 'Male')
    df['age_admit'] = np.where(df['age_admit'] >= 85, 85, df['age_admit'])

    return df

sepsis = data_clean(sepsis)
nonsepsis = data_clean(nonsepsis)

sepsis['Sepsis'] = 1
nonsepsis['Sepsis'] = 0
df_table1 = pd.concat((sepsis, nonsepsis), axis=0, ignore_index=True)

## Table 1 Descriptive statistics

In [4]:
columns = ['ETHNICITY', 'gender', 'INSURANCE', 'MARITAL_STATUS', 'los', 'age_admit']
categorical = ['ETHNICITY', 'gender', 'INSURANCE', 'MARITAL_STATUS']
order = {
    'ETHNICITY': ['White', 'Black', 'Hispanic', 'Asian', 'Other'],
    'MARITAL_STATUS': ['Single', 'Married', 'Divorced', 'Widowed', 'Other', 'Unknown']
    }
label = {
    'age_admit': 'Age (yrs) at first admission',
    'los': 'Length of stay (days)',
    'ETHNICITY': 'Race/Ethnicity',
    'MARITAL_STATUS': 'Marital status',
    'gender': 'Gender',
    'INSURANCE': 'Insurance'
}

t1 = TableOne(
    df_table1, 
    columns = columns, 
    categorical = categorical,
    nonnormal = ['los'],  
    groupby = 'Sepsis', 
    limit = 6, 
    order = order,
    pval = False,
    missing = False,
    rename = label
)

t1
# print(t1.tabulate(tablefmt="latex"))


Grouped by Sepsis                                 
                                                             Overall               0                1
n                                                              47871           42836             5035
Race/Ethnicity, n (%)                   White           33193 (69.3)    29530 (68.9)      3663 (72.8)
                                        Black             4066 (8.5)      3548 (8.3)       518 (10.3)
                                        Hispanic          1727 (3.6)      1563 (3.6)        164 (3.3)
                                        Asian             1733 (3.6)      1568 (3.7)        165 (3.3)
                                        Other            7152 (14.9)     6627 (15.5)       525 (10.4)
Gender, n (%)                           Female          20988 (43.8)    18748 (43.8)      2240 (44.5)
                                        Male            26883 (56.2)    24088 (56.2)      2795 (55.5)
Insurance, n (%)                        Government        1620 (3.4)      1501 (3.5)        119 (2.4)
                                        Medicaid          4624 (9.7)      4151 (9.7)        473 (9.4)
                                        Medicare        21270 (44.4)    18174 (42.4)      3096 (61.5)
                                        Private         19756 (41.3)    18441 (43.1)      1315 (26.1)
                                        Self Pay           601 (1.3)       569 (1.3)         32 (0.6)
Marital status, n (%)                   Single          10175 (21.3)     8840 (20.6)      1335 (26.5)
                                        Married         19100 (39.9)    16864 (39.4)      2236 (44.4)
                                        Divorced          2454 (5.1)      2115 (4.9)        339 (6.7)
                                        Widowed          5587 (11.7)     4782 (11.2)       805 (16.0)
                                        Other              431 (0.9)       364 (0.8)         67 (1.3)
                                        Unknown         10124 (21.1)     9871 (23.0)        253 (5.0)
Length of stay (days), median [Q1,Q3]                 7.0 [4.0,15.0]  7.0 [4.0,14.0]  17.0 [8.0,35.0]
Age (yrs) at first admission, mean (SD)                  52.7 (28.1)     51.2 (28.9)      65.4 (15.9)

## Feature creation

In [6]:
def feature_create(df):
    df_angus_sepsis = angus_sepsis.drop(columns=['hadm_id', 'explicit_sepsis', 'angus', 'infection']).drop_duplicates(ignore_index=True)
    df_sofa = sofa[['subject_id', 'SOFA']].groupby('subject_id')['SOFA'].max().to_frame().reset_index()
    df_sirs = sirs[['subject_id', 'sirs']].groupby('subject_id')['sirs'].max().to_frame().reset_index()

    df = df.merge(df_angus_sepsis, how='inner', on='subject_id') \
        .merge(df_sofa, how='inner', on='subject_id') \
        .merge(df_sirs, how='inner', on='subject_id')
    df = df.drop_duplicates('subject_id', ignore_index=True).drop(columns=['dob', 'dod', 'age_death', 'admit_time'])

    df = pd.concat((
        df.drop(columns=['gender', 'ETHNICITY', 'INSURANCE', 'MARITAL_STATUS']),
        pd.get_dummies(df['gender'], dummy_na=False, prefix='gender'), 
        pd.get_dummies(df['ETHNICITY'], dummy_na=False, prefix='ethnicity'),
        pd.get_dummies(df['MARITAL_STATUS'], dummy_na=False, prefix='marital_status'),
        pd.get_dummies(df['INSURANCE'], dummy_na=False, prefix='insurance')), axis = 1).reset_index(drop=True)
    
    
    df = df.merge(lab_measures, how='left', left_on='subject_id', right_on='SUBJECT_ID')
    df = df.fillna(df.median())
    
    mms = MinMaxScaler()
    df[df.columns] = mms.fit_transform(df)
    return df


## Model training and prediction

In [8]:
df_model = feature_create(df_table1)
X = df_model.drop(columns=['Sepsis', 'subject_id']).values
y = df_model['Sepsis'].values

print(f"Number of features included: {X.shape[1]}")

Number of features included: 61


In [9]:
skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
avg_auc = 0
feature_importance = np.zeros((X.shape[1], ))

for fold, (tr_idx, val_idx) in enumerate(skf.split(X, y)):
    tr_x, tr_y = X[tr_idx], y[tr_idx]
    val_x, val_y = X[val_idx], y[val_idx]
    model = GradientBoostingClassifier()
    model.fit(tr_x, tr_y)
    pred = model.predict_proba(val_x)[:, 1]
    fold_score = roc_auc_score(val_y, pred)
    
    feature_importance += model.feature_importances_ / skf.n_splits
    print(f"Fold - {fold} AUC: {fold_score:.5f}")
    avg_auc += fold_score / skf.n_splits
    
print(f'Average AUC: {avg_auc:.5f}')


Fold - 0 AUC: 0.89675
Fold - 1 AUC: 0.89958
Fold - 2 AUC: 0.90224
Fold - 3 AUC: 0.90438
Fold - 4 AUC: 0.90986
Average AUC: 0.90257


## Feature importance

In [14]:
pd.DataFrame(dict(
    var = df_model.drop(columns=['Sepsis', 'subject_id']).columns, 
    importance = feature_importance)).sort_values('importance', ascending=False, ignore_index=True).head(20)

,var,importance
0,SOFA,0.227286
1,organ_dysfunction,0.132561
2,urea_nitrogen_max,0.098024
3,creatinine_max,0.092864
4,sirs,0.076701
5,lactate_max,0.053800
6,los,0.042174
7,lactate_min,0.028582
8,WBC_max,0.025187
9,lactate_avg,0.023556


## Lightgbm CV

In [30]:
lgbm_param = {
        'num_leaves': hp.choice('num_leaves', np.arange(2, 21)),
        'learning_rate': hp.uniform('learning_rate', 0.005, 0.1),
        'feature_fraction': hp.uniform('feature_fraction', 0.01, 0.5),
        'max_depth': hp.choice('max_depth', np.arange(2, 11)),
        'objective': 'binary',
        # 'boosting_type': 'dart',
        'metric': 'auc',
        'verbose': -1,
        'device_type': 'gpu'
    }

def f_lgbm(params):
    tr_data = lgbm.Dataset(X, y)
    res = lgbm.cv(params, tr_data, num_boost_round=1000, early_stopping_rounds=100, seed=42)
    return {'loss': -np.mean(res['auc-mean']).round(5), 'status': STATUS_OK}
    # lgbm_pred = np.zeros((len(X), ))
    # auc = np.zeros(5)
    # for i, (tr_idx, te_idx) in enumerate(StratifiedKFold(5, shuffle=True, random_state=42).split(X, y)):
    #     if i > 0: break
    #     tr_data = lgbm.Dataset(X[tr_idx], y[tr_idx]) #, categorical_feature=cat_col)
    #     te_data = lgbm.Dataset(X[te_idx], y[te_idx])#, categorical_feature=cat_col)
    #     clf = lgbm.train(params,
    #                      tr_data,
    #                      num_boost_round=2000,
    #                      verbose_eval=False,
    #                      valid_sets=[tr_data, te_data],
    #                      early_stopping_rounds=100,
    #                 )
    #     lgbm_pred[te_idx] = clf.predict(X[te_idx], num_iteration=clf.best_iteration)
    #     auc[i] = roc_auc_score(y[te_idx], lgbm_pred[te_idx])
    #     del clf
    #     gc.collect()
        
    # return {'loss': -np.mean(auc[0]).round(5), 'status': STATUS_OK}

In [31]:
trials = Trials()
lgbm_best = fmin(f_lgbm, lgbm_param, algo=tpe.suggest, rstate=np.random.RandomState(42), max_evals=20, trials=trials)

100%|██████████| 20/20 [04:49<00:00, 14.47s/trial, best loss: -0.90635]
